In [1]:

import sys 
sys.path.append(r'C:\Users\sergiojesus\Desktop\Diogo2017\EQS_LOCAL\Bitbucket_eqs')
import analytics_stat.distributions as di

import warnings
warnings.filterwarnings('ignore')

import datetime
import scipy.stats as s
#import scipy.optimize as opt
#import scipy.special as special
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
%matplotlib inline
# %matplotlib notebook

import os
import json
import pandas as pd
import qgrid
from IPython.display import HTML, Image, display, clear_output
# display(HTML("<style>.container { width:95% !important; }</style>"))

# py.sign_in('diogo.antunes.goncalves', 'kt7T5JadNn2AhWk3DdPR')

import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

import traitlets
from tkinter import Tk, filedialog

display(HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
This report hides the raw code used for its implementation by default.
To hide/show the raw code, click <a href="javascript:code_toggle()">here</a>.'''))

display(HTML('''
<button id="do_run_all">Click to start</button>
<script>
$("#do_run_all").click(
    function () {
        $("#run_cell").click();
    }
);
</script>'''))
        


In [14]:
class Display:
    
    def __init__(self):
        
        # Select file button
        self._bsf = widgets.Button(layout=widgets.Layout(width='500px',height='50px'))
        self._bsf.add_traits(files=traitlets.traitlets.List())
        self._bsf.description = "Select File"
        self._bsf.icon = "square-o"
        self._bsf.on_click(self.select_excel)
        
        
        # Load Excel button
        self._blj = widgets.Button(layout=widgets.Layout(width='500px',height='50px'))
        self._blj.description = "Load File"
        self._blj.icon = 'download'
        self._blj.on_click(self.load_excel)
        self._blj.disabled = True
                
        # Spreadsheet Dropdown
        self._dds = widgets.Dropdown(options=['Select file'], value='Select file', disabled=True)
        
        # Column Selection
        self._csm = widgets.SelectMultiple(options=['Select table'], value =['Select table'], disabled=True)
        
        # Column Selection button
        self._bcs = widgets.Button(layout=widgets.Layout(width='500px',height='50px'))
        self._bcs.description = "Choose Columns"
        self._bcs.on_click(self.choose_columns)
        self._bcs.disabled = True
        
        # define layout
        box_layout = widgets.Layout(display='center',
                            flex_flow='row',
                            align_items='center',
                            border='None',
                            justify_content='space-between',
                            width='800px')
        
        self._box = widgets.Box(children=[self._bsf, widgets.Label('Spreadsheet:'), self._dds, self._blj,
                                          self._csm], layout=box_layout)
        display(self._box)
            
    def select_excel(self,b): 
        # Create Tk root
        root = Tk()
        # Hide the main window
        root.withdraw()
        # Raise the root to the top of all windows.
        root.call('wm', 'attributes', '.', '-topmost', True)
        
        new_file = filedialog.askopenfilename(multiple=True,filetypes = [("XLS",'*.xls')])
        if new_file != "":
            self._bsf.files = new_file
            self._bsf.description = "File Selected"
            self._bsf.icon = "check-square-o"
            self._bsf.button_style = "success"
            self._blj.disabled = False
            print('Selected file: {}'.format(self._bsf.files))
            file_path = self._bsf.files
            self._excel = pd.ExcelFile(file_path[0], on_demand = True)
            self._sheets = self._excel.sheet_names
            self._dds.options = self._sheets
            self._dds.disabled = False

    def load_excel(self, b):

        self._dataframe = pd.read_excel(self._bsf.files[0], sheet_name = self._dds.value) 
        self._blj.button_style = "success"
        display(self._dataframe.head())
        self._csm.disabled = False
        self._csm.options = list(self._dataframe.columns.values)
        

In [15]:
d = Display()

Box(children=(Button(description='Select File', icon='square-o', layout=Layout(height='50px', width='500px'), …

Selected file: ["C:/Users/sergiojesus/Desktop/Recursos/Dados/Incidentes e OT's - Pontes 114 a 146.2 MX6 e MX7.xls"]


,OT,Descrição OT,Relevante,recordkey,worklogid,Column1,Descrição Log,Descrição Longa Log,Estado,Tipo Trabalho,...,Activo,Descrição Ativo,Data Registo,Ano,Mês,Sector,Inío Intervenção,Reposição do Activo,Fim Intervenção,Incidendia
0,ALS100011,Ponte 144 em avaria e com falha de autolevel,n,ALS100011,25902.0,25902.0,NaN,Foi verificada a UPS e no local está sem alarm...,FECHADA,MC,...,32124V010020,Ponte Telescópica FP-TEK 144,2010-07-26,2010,7,ALSENGMAN-CTC-TME,2010-07-26 07:35:00.000,2010-07-26 07:50:00.000,2010-07-26 07:50:00.000,1014.0
1,ALS100048,varia da manga 146,n,ALS100048,21676.0,21676.0,safty edge actuado. recolocada a manga ficando...,Depois de verificações a varios sistemas de se...,FECHADA,MC,...,32124V010022,Ponte Telescópica FP-TEK 146.1,2010-07-26,2010,7,ALSENGMAN-CTC-TME,2010-07-26 18:15:00.000,2010-07-26 18:30:00.000,2010-07-26 18:32:00.000,1042.0
2,ALS100053,Avaria manga 143,n,ALS100053,21768.0,NaN,puxada a manga atrás e retirado o alarme fican...,NaN,FECHADA,MC,...,32124V010019,Ponte Telescópica FP-TEK 143,2010-07-27,2010,7,ALSENGMAN-CTC-TME,2010-07-27 07:33:00.000,2010-07-27 07:45:00.000,2010-07-27 07:45:00.000,1059.0
3,ALS1000715,Reporte de ponte 146.2 fora de parque,n,ALS1000715,179442.0,179442.0,Reporte de ponte 146.2 fora de parque,Foram verificadas as duas pontes 146.1 e146.2 ...,REALIZ,MC,...,32124V010023,Ponte Telescópica FP-TEK 146.2,2012-11-21,2012,11,ALSCMT-PSE,2012-11-21 07:22:00.000,NaN,2012-11-21 07:32:00.000,42300.0
4,ALS100100,Avaria Ponte Telescópica FP-TEK 146.2,n,ALS100100,21774.0,21774.0,o operador tinha dificuldade em manobrar a manga,foi dada alguma informação de como proceder fi...,FECHADA,MC,...,32124V010023,Ponte Telescópica FP-TEK 146.2,2010-07-27,2010,7,ALSENGMAN-CTC-TME,2010-07-27 12:00:00.000,2010-07-27 12:20:00.000,2010-07-27 12:20:00.000,1073.0


In [4]:
DF = pd.read_excel(r"C:\Users\sergiojesus\Desktop\Recursos\Dados\Incidentes e OT's - Pontes 114 a 146.2 MX6 e MX7.xls",
                   sheet_name = "OT's MX7 até 11122012")

In [5]:

print(DF)

              OT                                       Descrição OT Relevante  \
0      ALS100011       Ponte 144 em avaria e com falha de autolevel         n   
1      ALS100048                                 varia da manga 146         n   
2      ALS100053                                   Avaria manga 143         n   
3     ALS1000715              Reporte de ponte 146.2 fora de parque         n   
4      ALS100100              Avaria Ponte Telescópica FP-TEK 146.2         n   
5      ALS100100              Avaria Ponte Telescópica FP-TEK 146.2         n   
6      ALS100111                          ALARME GERAL NA MANGA 145         n   
7      ALS100113                          ALARME GERAL NA PONTE 142         n   
8      ALS100113                          ALARME GERAL NA PONTE 142         n   
9      ALS100135                            Manga 146  com anomalia         n   
10     ALS100142                Camara da manga virada ao contrario         n   
11     ALS100142            